# Data preparation

Features
- Modality (01 = full-AV, 02 = video-only, 03 = audio-only).
- Vocal channel (01 = speech, 02 = song).
- Emotion (01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised).
- Emotional intensity (01 = normal, 02 = strong). NOTE: There is no strong intensity for the 'neutral' emotion.
- Statement (01 = "Kids are talking by the door", 02 = "Dogs are sitting by the door").
- Repetition (01 = 1st repetition, 02 = 2nd repetition).
- Actor (01 to 24. Odd numbered actors are male, even numbered actors are female).

NB We considered only the speech videos (vocal channel=01) with both audio and video (modality=01)

In [3]:
emotions = {1:'neutral', 2:'calm', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'}
emotional_intensity = {1:'normal', 2:'strong'}

In [4]:
import re
import os
import pandas as pd
import cv2
import random
import numpy as np
from tensorflow import keras
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from tqdm import tqdm

In [5]:
path = "Datasets/RAVDESS/"

In [6]:
filenames = []
feats = []
labels = []
paths = []

for (dirpath, dirnames, fn) in os.walk(path):
    for name in fn:
        filename = name.split('.')[0]
        feat = filename.split('-')[2:]
        label = feat[0]
        filenames.append(filename)
        feats.append(feat)
        labels.append(label)
        paths.append(dirpath + '/' + filename)
        
filenames[:5]

['01-01-01-01-01-01-01',
 '01-01-01-01-01-02-01',
 '01-01-01-01-02-01-01',
 '01-01-01-01-02-02-01',
 '01-01-02-01-01-01-01']

## Data Exploration

In [5]:
df = pd.DataFrame(feats, columns = ['emotion', 'emotional intensity', 'statement', 'repetition', 'actor']).astype(int)

df['emotion'] = df['emotion'].map(emotions)
df['emotional intensity'] = df['emotional intensity'].map(emotional_intensity)

df['index'] = filenames
df.set_index('index', inplace=True)

df

,emotion,emotional intensity,statement,repetition,actor
index,,,,,
01-01-01-01-01-01-01,neutral,normal,1,1,1
01-01-01-01-01-02-01,neutral,normal,1,2,1
01-01-01-01-02-01-01,neutral,normal,2,1,1
01-01-01-01-02-02-01,neutral,normal,2,2,1
01-01-02-01-01-01-01,calm,normal,1,1,1
...,...,...,...,...,...
01-01-08-01-02-02-24,surprise,normal,2,2,24
01-01-08-02-01-01-24,surprise,strong,1,1,24
01-01-08-02-01-02-24,surprise,strong,1,2,24


## Export frames

- one frame every skip=3 starting from the 21th frame
- proportional resize to obtain height=224
- saved as png with and name videoname_iframe

### 398x224 normal

In [ ]:
def prepare_all_videos(filenames, paths, skip=1):
    nframes_tot = 0
    
    for count, video in enumerate(zip(filenames, paths)):
        # Gather all its frames
        save_frames(video[0], video[1], video[1].replace('RAVDESS', 'RAVDESS_frames'), skip)
        print(f"Processed videos {count+1}/{len(paths)}")
    return


def save_frames(filename, input_path, output_path, skip):
    # Initialize video reader
    cap = cv2.VideoCapture(input_path + '.mp4')
    frames = []
    count = 0
    
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    try:
        # Loop through all frames
        while True:
            # Capture frame
            ret, frame = cap.read()
            if (count % skip == 0 and count > 20):
                #print(frame.shape)
                if not ret:
                    break
                frame = cv2.resize(frame, (398, 224))
                cv2.imwrite(output_path + '/' + f'{filename}_{count}' + '.png', frame)
            count += 1
    finally:
        cap.release()
    return

In [ ]:
prepare_all_videos(filenames, paths, skip=3)

### 224x224 black background

In [ ]:
def prepare_all_videos(filenames, paths, skip=1):
    nframes_tot = 0
    
    for count, video in enumerate(zip(filenames, paths)):
        # Gather all its frames
        save_frames(video[0], video[1], video[1].replace('RAVDESS', 'RAVDESS_frames_black'), skip)
        print(f"Processed videos {count+1}/{len(paths)}")
    return


def save_frames(filename, input_path, output_path, skip):
    # Initialize video reader
    cap = cv2.VideoCapture(input_path + '.mp4')
    frames = []
    count = 0
    
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    try:
        # Loop through all frames
        while True:
            # Capture frame
            ret, frame = cap.read()
            if (count % skip == 0 and count > 20):
                #print(frame.shape)
                if not ret:
                    break
                #####
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)                  # background from white to black
                ret, thresh = cv2.threshold(gray, 220, 255, cv2.THRESH_BINARY)
                frame[thresh == 255] = 0
                #####
                frame = cv2.resize(frame, (398, 224))
                frame = frame[0:224, 87:311]
                cv2.imwrite(output_path + '/' + f'{filename}_{count}' + '.png', frame)
            count += 1
    finally:
        cap.release()
    return

In [ ]:
prepare_all_videos(filenames, paths, skip=3)

### 224x224 only faces BW

In [5]:
def prepare_all_videos(filenames, paths, skip=1):
    nframes_tot = 0
    
    for count, video in enumerate(zip(filenames, paths)):
        # Gather all its frames
        save_frames(video[0], video[1], video[1].replace('RAVDESS', 'RAVDESS_frames_face_BW'), skip)
        print(f"Processed videos {count+1}/{len(paths)}")
    return


def save_frames(filename, input_path, output_path, skip):
    # Initialize video reader
    cap = cv2.VideoCapture(input_path + '.mp4')
    haar_cascade = cv2.CascadeClassifier('./Other/haarcascade_frontalface_default.xml')
    frames = []
    count = 0
    
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    try:
        # Loop through all frames
        while True:
            # Capture frame
            ret, frame = cap.read()
            if (count % skip == 0 and count > 20):
                #print(frame.shape)
                if not ret:
                    break
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                faces = haar_cascade.detectMultiScale(frame, scaleFactor=1.12, minNeighbors=9)
                # if len(faces) != 1:
                    
                if len(faces) == 0:
                    faces = haar_cascade.detectMultiScale(frame, scaleFactor=1.02, minNeighbors=9)
                    if len(faces) == 0:
                        raise Exception(f"Still no faces {len(faces)} {filename}")
                if len(faces) > 1:
                    ex = []
                    print(type(faces))
                    for elem in faces:
                        for (x, y, w, h) in [elem]:
                            ex.append(frame[y:y + h, x:x + w])

                    print(filename)
                    plt.figure()
                    f, axarr = plt.subplots(4,1)
                    axarr[0].imshow(ex[0])
                    axarr[1].imshow(ex[1])
                    plt.show()

                    inp = int(input())
                    faces = [faces[inp]]
                #     raise Exception(f"More than 1 faces detected in {filename}")

                for (x, y, w, h) in faces:
                    face = frame[y:y + h, x:x + w]

                face = cv2.resize(face, (234, 234))
                face = face[5:-5, 5:-5]
                cv2.imwrite(output_path + '/' + f'{filename}_{count}' + '.png', face)
            count += 1
    finally:
        cap.release()
    return

In [ ]:
prepare_all_videos(filenames, paths, skip=3)

### PCA 50 components

In [7]:
emotions_tras = {1:1, 2:4, 3:5, 4:0, 5:3, 6:2, 7:6}
emotions = {0:'angry', 1:'calm', 2:'disgust', 3:'fear', 4:'happy', 5:'sad', 6:'surprise'}

# emotions = {0:'neutral', 1:'calm', 2:'happy', 3:'sad', 4:'angry', 5:'fear', 6:'disgust', 7:'surprise'}
# emotional_intensity = {1:'normal', 2:'strong'}

dataset_path = "Datasets/RAVDESS_frames_face_BW/"
output_path = "Datasets/RAVDESS_frames_face_BW_SVD50/"

n_components = 50

height_orig = 224
width_orig = 224
height_targ = 112
width_targ = 112

batch_size = 64
num_classes = len(emotions)

val_actors = ['21', '22']
test_actors = ['23', '24']

In [ ]:
filenames_train = [] # train
filenames_val = [] # validation

for (dirpath, dirnames, fn) in os.walk(dataset_path):
    if fn != []:
        class_temp = int(fn[0].split('-')[2]) - 1
        if class_temp != 0:                                                     # exclude 'neutral' label
            if any(act in dirpath for act in (test_actors+val_actors))==False:  # select only train actors
                path = [os.path.join(dirpath, elem) for elem in fn]
                label = [emotions_tras[class_temp]] * len(fn)                   # emotion transposition
                filenames_train.append(list(zip(path, label)))
            
            if any(act in dirpath for act in val_actors):                       # select only validation actors
                path = [os.path.join(dirpath, elem) for elem in fn]
                label = [emotions_tras[class_temp]] * len(fn)
                filenames_val.append(list(zip(path, label)))

In [ ]:
def sampling(list, num_frames_desired):
    tot = []
    for elem in list:
        sampled_list = random.sample(elem, num_frames_desired)
        tot += sampled_list
    return(tot)


def load_dataset(filenames, n_components):
    frames_per_vid = min([len(elem) for elem in filenames])     # number of frames per clip in order to have balanced classes
    print("frames per video:", frames_per_vid) 

    filenames_sampled = sampling(filenames, frames_per_vid)
    random.shuffle(filenames_sampled)

    faces = []
    y = []


    for path, label in tqdm(filenames_sampled):
        face = cv2.imread(path)
        face = cv2.resize(face, (width_targ, height_targ))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        
        faces.append(face)
        y.append(label)
    
    # plt.imshow(x[1], cmap='gray')
    # plt.axis('off')
    # plt.show()

    faces = np.array(faces)

    avgFace = np.average(faces, axis=0)
    print('mean of faces')
    plt.imshow(avgFace, cmap='gray')
    plt.axis('off')
    plt.show()

    print(faces.shape)
    mean = np.mean(faces, axis=0)
    faces = [face - mean for face in faces]

    print('face post')
    plt.imshow(faces[0], cmap='gray')
    plt.axis('off')
    plt.show()

    faces = np.array(faces)
    # faces = np.reshape(faces, (100, ))
    faces = faces.reshape((faces.shape[0], faces.shape[1]**2))
    print(faces.shape)

    #now calculate eigen values and eigen vectors for cov_mat
    # cov_mat = np.cov(faces, rowvar = False)
    # print('computer covariace, now eig')

    # eig_values, eig_vectors = np.linalg.eigh(cov_mat)
    # print('computed eig, now sorting and cropping')

    # #sort the eigenvalues in descending order
    # sorted_index = np.argsort(eig_values)[::-1]
    
    # sorted_eigenvalue = eig_values[sorted_index]
    # #similarly sort the eigenvectors 
    # sorted_eigenvectors = eig_vectors[:,sorted_index]

    
    # eigenvector_subset = sorted_eigenvectors[:,0:n_components]
    # print(eigenvector_subset.shape)
    pca = PCA(n_components) # we need 2 principal components.
    x_reduced = pca.fit_transform(faces)
    
    # converted_data.shape


    # x_reduced = []
    # for face in faces:

    #     x_reduced = np.dot(eigenvector_subset.transpose(),face.transpose()).transpose()
    #     print(x_reduced.shape)

    # names = tf.data.Dataset.from_tensor_slices(names)
    # images = names.map(parse_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)

    # labels = [elem for elem in labels]
    # labels = tf.data.Dataset.from_tensor_slices(labels)

    # ds = tf.data.Dataset.zip((images, labels))
    # ds = configure_for_performance(ds)

    frame_number = len(filenames_sampled)
    step_per_epoch = frame_number // batch_size
    print('frames number:', frame_number, '\nbatch size:', batch_size, '\nbatch number:', step_per_epoch)
    return x_reduced, y, step_per_epoch

In [ ]:
train_ds, y, step_per_epoch_train = load_dataset(filenames_train[:80], n_components)